In [2]:
"""
    边缘检测：
    -1, -1, -1 
    -1,  8, -1
    -1, -1, -1
    锐化:
     0, -1,  0
    -1,  5, -1
     0, -1,  0
    高斯模糊：
    (1/16) 1, 2, 1
           2, 4, 2
           1, 2, 1

    一维卷积可以处理：文本、语言、时序序列
    三维可以：视频、医学图像、气象地图 
"""
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i+h, j: j+w] * K).sum()
    return Y

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

二维卷积层的时间复杂度：

输入X：$ c_i \times n_h \times n_w $

核W：$ c_o \times c_i \times k_h \times k_w $

偏差B：$ c_o \times c_i $

输出Y：$ c_o \times m_h \times m_w $

计算复杂度（浮点计算数FLOP）：$ O(c_i c_o k_h k_w m_h m_w) $ 

In [5]:
# 二维卷积层
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [6]:
# 检测图像中不同颜色的边缘
X = torch.ones((6, 8))
X[:, 2:6] = 0
X # 这张图片，有两种颜色，1，0

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [7]:
K = torch.tensor([[1.0, -1.0]]) # 这样只能检测竖向的

In [8]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [9]:
corr2d(X.t(), K) # 正面前面的K只能检测竖向的变化

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [10]:
# 学习由X生成Y的卷积核，可以这样去学习应该用什么卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

# 通道、批量大小、长、宽 
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'batch {i+1}, loss {l.sum():.3f}')

batch 2, loss 13.639
batch 4, loss 4.159
batch 6, loss 1.464
batch 8, loss 0.560
batch 10, loss 0.222


In [11]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9392, -1.0356]])